In [1]:
from kiwipiepy import Kiwi
from rank_bm25 import BM25Okapi
from typing import List, Dict
from tqdm import tqdm
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance, 
    VectorParams, 
    SparseVectorParams,
    SparseIndexParams,
    PointStruct,
    SparseVector,
    NamedVector,
    NamedSparseVector
)
from langchain_core.documents import Document
import json
import os
import hashlib
import numpy as np
from datetime import datetime


# ============================================
# 1. Kiwi BM25 인코더
# ============================================

class KiwiBM25Encoder:
    """Kiwi 형태소 분석기 기반 BM25 Sparse 임베딩"""
    
    def __init__(self, documents: List[str] = None, load_from_dict: dict = None):
        if load_from_dict:
            self.tokenized_corpus = load_from_dict['tokenized_corpus']
            self.vocabulary = load_from_dict['vocabulary']
            self.bm25 = BM25Okapi(self.tokenized_corpus)
            self.kiwi = Kiwi()
            self._build_idf_cache()
            print(f"✅ BM25 인코더 로드 완료 (Vocabulary: {len(self.vocabulary)})")
        else:
            print("🥝 Kiwi 초기화 중...")
            self.kiwi = Kiwi()
            print("✅ Kiwi 초기화 성공!")
            
            print("📝 형태소 분석 중...")
            self.tokenized_corpus = []
            for doc in tqdm(documents, desc="토크나이징"):
                tokens = self._tokenize(doc)
                self.tokenized_corpus.append(tokens)
            
            print("🔍 BM25 인덱스 생성 중...")
            self.bm25 = BM25Okapi(self.tokenized_corpus)
            
            self.vocabulary = {}
            idx = 0
            for doc_tokens in self.tokenized_corpus:
                for token in doc_tokens:
                    if token not in self.vocabulary:
                        self.vocabulary[token] = idx
                        idx += 1
            
            self._build_idf_cache()
            print(f"✅ Vocabulary 크기: {len(self.vocabulary)}")
    
    def _build_idf_cache(self):
        """각 토큰의 IDF 값을 미리 계산"""
        self.idf_cache = {}
        total_docs = len(self.tokenized_corpus)
        
        for token, idx in self.vocabulary.items():
            doc_freq = sum(1 for doc in self.tokenized_corpus if token in doc)
            idf = np.log((total_docs - doc_freq + 0.5) / (doc_freq + 0.5) + 1.0)
            self.idf_cache[idx] = max(0.0, idf)
    
    def _tokenize(self, text: str) -> List[str]:
        """텍스트를 형태소 분석"""
        result = self.kiwi.tokenize(text)
        tokens = []
        for token in result:
            if token.tag in ['NNG', 'NNP', 'VV', 'VA', 'MAG', 'SL']:
                tokens.append(token.form)
        return tokens
    
    def encode_query(self, query: str) -> Dict[int, float]:
        """쿼리를 sparse vector로 변환"""
        tokenized_query = self._tokenize(query)
        
        if not tokenized_query:
            return {}
        
        token_freq = {}
        for token in tokenized_query:
            if token in self.vocabulary:
                token_freq[token] = token_freq.get(token, 0) + 1
        
        if not token_freq:
            return {}
        
        sparse_vector = {}
        for token, freq in token_freq.items():
            idx = self.vocabulary[token]
            idf = self.idf_cache.get(idx, 0.0)
            
            tf = freq / len(tokenized_query)
            score = tf * idf
            
            if score > 0:
                sparse_vector[idx] = float(score)
        
        return sparse_vector
    
    def encode_document(self, doc: str, doc_idx: int = None) -> Dict[int, float]:
        """문서를 sparse vector로 변환"""
        tokens = self._tokenize(doc)
        
        if not tokens:
            return {}
        
        tf = {}
        for token in tokens:
            if token in self.vocabulary:
                idx = self.vocabulary[token]
                tf[idx] = tf.get(idx, 0.0) + 1.0
        
        if not tf:
            return {}
        
        max_tf = max(tf.values())
        sparse_vector = {k: v / max_tf for k, v in tf.items()}
        
        return sparse_vector
    
    def save(self, filepath: str):
        """JSON으로 저장"""
        data = {
            'tokenized_corpus': self.tokenized_corpus,
            'vocabulary': self.vocabulary
        }
        
        json_filepath = filepath.replace('.pkl', '.json')
        with open(json_filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        print(f"💾 BM25 인코더 저장: {json_filepath}")
        return json_filepath
    
    @staticmethod
    def load(filepath: str):
        """JSON에서 로드"""
        json_filepath = filepath.replace('.pkl', '.json')
        
        if not os.path.exists(json_filepath):
            raise FileNotFoundError(f"파일을 찾을 수 없습니다: {json_filepath}")
        
        with open(json_filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        print(f"📂 BM25 인코더 로드: {json_filepath}")
        return KiwiBM25Encoder(load_from_dict=data)


# ============================================
# 2. 하이브리드 검색 엔진 클래스
# ============================================
import time
class HybridSearchEngine:
    """
    Sparse (Kiwi BM25) + Dense (Clova) 하이브리드 검색 엔진
    """
    
    def __init__(
        self,
        clova_embedder,
        client: QdrantClient,
        collection_name: str = "legal_docs_hybrid",
        bm25_encoder_file: str = "kiwi_bm25_encoder.pkl"
    ):
        """
        Args:
            clova_embedder: Clova 임베더
            client: Qdrant 클라이언트
            collection_name: 컬렉션 이름
            bm25_encoder_file: BM25 인코더 저장 파일
        """
        self.clova_embedder = clova_embedder
        self.client = client
        self.collection_name = collection_name
        self.bm25_encoder_file = bm25_encoder_file
        self.bm25_encoder = None
        
        print("🔧 HybridSearchEngine 초기화 완료")
    
    def index(self, documents: List[Document], force_rebuild: bool = False):
        """
        문서 인덱싱
        
        Args:
            documents: 인덱싱할 Document 리스트
            force_rebuild: True면 강제로 재생성
        """
        print("\n" + "="*80)
        print("🚀 하이브리드 인덱싱 시작")
        print("="*80)
        
        # ========================================
        # 1단계: BM25 인코더 준비
        # ========================================
        print("\n1️⃣ Kiwi BM25 인코더 준비 중...")
        
        corpus = [doc.page_content for doc in documents]
        json_file = self.bm25_encoder_file.replace('.pkl', '.json')
        
        if os.path.exists(json_file) and not force_rebuild:
            print("  📂 기존 BM25 인코더 로드")
            self.bm25_encoder = KiwiBM25Encoder.load(self.bm25_encoder_file)
        else:
            print("  🆕 새 BM25 인코더 생성")
            self.bm25_encoder = KiwiBM25Encoder(corpus)
            self.bm25_encoder.save(self.bm25_encoder_file)
        
        # ========================================
        # 2단계: Qdrant 컬렉션 생성
        # ========================================
        print(f"\n2️⃣ Qdrant 컬렉션 '{self.collection_name}' 생성 중...")
        
        try:
            self.client.delete_collection(self.collection_name)
            print("  🗑️  기존 컬렉션 삭제")
        except:
            pass
        
        self.client.create_collection(
            collection_name=self.collection_name,
            vectors_config={
                "dense": VectorParams(size=1024, distance=Distance.COSINE)
            },
            sparse_vectors_config={
                "sparse": SparseVectorParams(
                    index=SparseIndexParams(on_disk=False)
                )
            }
        )
        print("  ✅ 컬렉션 생성 완료")
        
        # ========================================
        # 3단계: 문서 인덱싱
        # ========================================
        print(f"\n3️⃣ {len(documents)}개 문서 인덱싱 중...")
        print("    (각 문서: Clova 임베딩 → Kiwi BM25 → Qdrant 저장)")
        
        points = []
        batch_size = 50
        
        for idx, doc in enumerate(tqdm(documents, desc="인덱싱")):
            # Dense 임베딩 (Clova - API 호출)
            dense_vector = self.clova_embedder.embed_query(doc.page_content)
            time.sleep(1)
            # Sparse 임베딩 (Kiwi - 로컬)
            sparse_vector = self.bm25_encoder.encode_document(doc.page_content)
            
            if sparse_vector:
                sparse_vec = SparseVector(
                    indices=list(sparse_vector.keys()),
                    values=list(sparse_vector.values())
                )
            else:
                sparse_vec = SparseVector(indices=[], values=[])
            
            point = PointStruct(
                id=idx,
                vector={
                    "dense": dense_vector,
                    "sparse": sparse_vec
                },
                payload={
                    "page_content": doc.page_content,
                    "metadata": doc.metadata,
                    "doc_id": doc.id
                }
            )
            points.append(point)
            
            if len(points) >= batch_size:
                self.client.upsert(collection_name=self.collection_name, points=points)
                points = []
        
        if points:
            self.client.upsert(collection_name=self.collection_name, points=points)
        
        print(f"\n" + "="*80)
        print("✅ 인덱싱 완료!")
        print("="*80)
        print(f"📊 통계:")
        print(f"  - 문서 수: {len(documents)}개")
        print(f"  - Vocabulary: {len(self.bm25_encoder.vocabulary)}개")
        print(f"  - 컬렉션: {self.collection_name}")
        print("="*80)

    def add_documents(self, new_documents: List[Document], skip_duplicates: bool = True):
        """
        기존 컬렉션에 새 문서 추가
        
        Args:
            new_documents: 추가할 Document 리스트
            skip_duplicates: True면 중복 문서 건너뛰기
        """
        print("\n" + "="*80)
        print("➕ 문서 추가 시작")
        print("="*80)
        
        # BM25 인코더 로드
        if self.bm25_encoder is None:
            json_file = self.bm25_encoder_file.replace('.pkl', '.json')
            if not os.path.exists(json_file):
                raise ValueError(
                    "BM25 인코더가 없습니다. 먼저 index()를 실행하세요."
                )
            print("📂 BM25 인코더 로드 중...")
            self.bm25_encoder = KiwiBM25Encoder.load(self.bm25_encoder_file)
        
        # 컬렉션 존재 확인
        collections = self.client.get_collections().collections
        if not any(c.name == self.collection_name for c in collections):
            raise ValueError(
                f"컬렉션 '{self.collection_name}'이 존재하지 않습니다. "
                "먼저 index()를 실행하세요."
            )
        
        # 현재 최대 ID 및 기존 문서 확인
        collection_info = self.client.get_collection(self.collection_name)
        current_max_id = collection_info.points_count
        
        # 중복 체크
        if skip_duplicates:
            print("\n🔍 중복 문서 체크 중...")
            
            # 기존 문서의 doc_id 수집
            existing_doc_ids = set()
            scroll_result = self.client.scroll(
                collection_name=self.collection_name,
                limit=10000,  # 한 번에 가져올 최대 개수
                with_payload=True
            )
            
            for point in scroll_result[0]:
                if 'doc_id' in point.payload:
                    existing_doc_ids.add(point.payload['doc_id'])
            
            print(f"  📋 기존 문서 ID: {len(existing_doc_ids)}개")
            
            # 새 문서 필터링
            filtered_documents = []
            duplicate_count = 0
            
            for doc in new_documents:
                doc_id = doc.id if hasattr(doc, 'id') else None
                
                if doc_id and doc_id in existing_doc_ids:
                    duplicate_count += 1
                else:
                    filtered_documents.append(doc)
            
            print(f"  ✅ 중복 제외: {duplicate_count}개")
            print(f"  ➕ 추가할 문서: {len(filtered_documents)}개")
            
            new_documents = filtered_documents
        
        if not new_documents:
            print("\n⚠️  추가할 새 문서가 없습니다.")
            return
        
        print(f"\n📊 현재 상태:")
        print(f"  - 기존 문서 수: {current_max_id}개")
        print(f"  - 추가할 문서 수: {len(new_documents)}개")
        
        # 새 문서 인덱싱
        print(f"\n📝 {len(new_documents)}개 문서 추가 중...")
        
        points = []
        batch_size = 50
        
        for idx, doc in enumerate(tqdm(new_documents, desc="추가")):
            new_id = current_max_id + idx
            
            # Dense 임베딩
            dense_vector = self.clova_embedder.embed_query(doc.page_content)
            time.sleep(1)
            
            # Sparse 임베딩
            sparse_vector = self.bm25_encoder.encode_document(doc.page_content)
            
            if sparse_vector:
                sparse_vec = SparseVector(
                    indices=list(sparse_vector.keys()),
                    values=list(sparse_vector.values())
                )
            else:
                sparse_vec = SparseVector(indices=[], values=[])
            
            point = PointStruct(
                id=new_id,
                vector={
                    "dense": dense_vector,
                    "sparse": sparse_vec
                },
                payload={
                    "page_content": doc.page_content,
                    "metadata": doc.metadata,
                    "doc_id": doc.id if hasattr(doc, 'id') else str(new_id)
                }
            )
            points.append(point)
            
            if len(points) >= batch_size:
                self.client.upsert(collection_name=self.collection_name, points=points)
                points = []
        
        if points:
            self.client.upsert(collection_name=self.collection_name, points=points)
        
        # 최종 상태
        final_info = self.client.get_collection(self.collection_name)
        
        print(f"\n" + "="*80)
        print("✅ 문서 추가 완료!")
        print("="*80)
        print(f"📊 최종 상태:")
        print(f"  - 전체 문서 수: {final_info.points_count}개")
        print(f"  - 실제 추가된 문서: {len(new_documents)}개")
        print("="*80)

    def search(
        self,
        query: str,
        top_k: int = 5,
        dense_weight: float = 0.7,
        sparse_weight: float = 0.3,
        verbose: bool = True
    ):
        """
        하이브리드 검색
        
        Args:
            query: 검색 쿼리
            top_k: 반환할 결과 수
            dense_weight: Dense 가중치
            sparse_weight: Sparse 가중치
            verbose: 상세 정보 출력
        
        Returns:
            검색 결과 리스트
        """
        # BM25 인코더 확인
        if self.bm25_encoder is None:
            json_file = self.bm25_encoder_file.replace('.pkl', '.json')
            if os.path.exists(json_file):
                print("📂 BM25 인코더 로드 중...")
                self.bm25_encoder = KiwiBM25Encoder.load(self.bm25_encoder_file)
            else:
                raise ValueError("BM25 인코더가 없습니다. 먼저 index()를 실행하세요.")
        
        if verbose:
            print(f"\n{'='*80}")
            print(f"🔍 검색 쿼리: '{query}'")
            print(f"{'='*80}")
        
        # ========================================
        # 1단계: Dense 쿼리 벡터 (Clova)
        # ========================================
        if verbose:
            print("\n1️⃣ Dense 벡터 생성 (Clova)...")
        
        dense_query = self.clova_embedder.embed_query(query)
        
        if verbose:
            print(f"  ✅ Dense 벡터: {len(dense_query)} 차원")
        
        # ========================================
        # 2단계: Sparse 쿼리 벡터 (Kiwi BM25)
        # ========================================
        if verbose:
            print("\n2️⃣ Sparse 벡터 생성 (Kiwi BM25)...")
        
        sparse_query = self.bm25_encoder.encode_query(query)
        
        if verbose:
            tokens = self.bm25_encoder._tokenize(query)
            print(f"  📝 형태소: {tokens}")
            print(f"  ✅ Sparse 벡터: {len(sparse_query)} 개 (non-zero)")
        
        # ========================================
        # 3단계: Dense 검색
        # ========================================
        if verbose:
            print(f"\n3️⃣ Dense 검색 (가중치: {dense_weight})...")
        
        dense_results = self.client.search(
            collection_name=self.collection_name,
            query_vector=NamedVector(name="dense", vector=dense_query),
            limit=top_k * 3,
            with_payload=True
        )
        
        if verbose:
            print(f"  ✅ {len(dense_results)}개 결과")
        
        # ========================================
        # 4단계: Sparse 검색
        # ========================================
        if sparse_query:
            if verbose:
                print(f"\n4️⃣ Sparse 검색 (가중치: {sparse_weight})...")
            
            sparse_query_vec = NamedSparseVector(
                name="sparse",
                vector=SparseVector(
                    indices=list(sparse_query.keys()),
                    values=list(sparse_query.values())
                )
            )
            
            sparse_results = self.client.search(
                collection_name=self.collection_name,
                query_vector=sparse_query_vec,
                limit=top_k * 3,
                with_payload=True
            )
            
            if verbose:
                print(f"  ✅ {len(sparse_results)}개 결과")
        else:
            sparse_results = []
            if verbose:
                print(f"\n4️⃣ Sparse 검색 건너뜀 (유효 토큰 없음)")
        
        # ========================================
        # 5단계: RRF로 결과 결합
        # ========================================
        if verbose:
            print(f"\n5️⃣ RRF로 결과 결합...")
        
        rrf_scores = {}
        k = 10
        
        for rank, point in enumerate(dense_results, 1):
            rrf_scores[point.id] = rrf_scores.get(point.id, 0) + (dense_weight / (k + rank))
        
        for rank, point in enumerate(sparse_results, 1):
            rrf_scores[point.id] = rrf_scores.get(point.id, 0) + (sparse_weight / (k + rank))
        
        sorted_ids = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
        
        if not sorted_ids:
            if verbose:
                print("  ⚠️  검색 결과 없음")
            return []
        
        result_ids = [id for id, _ in sorted_ids]
        final_results = self.client.retrieve(
            collection_name=self.collection_name,
            ids=result_ids,
            with_payload=True
        )
        
        score_map = dict(sorted_ids)
        results_with_scores = []

        for point in final_results:
            result_dict = {
                'id': point.id,
                'score': score_map[point.id],
                'payload': point.payload
            }
            results_with_scores.append(result_dict)

        # score 기준으로 정렬
        results_with_scores.sort(key=lambda x: x['score'], reverse=True)

        if verbose:
            print(f"  ✅ 최종 {len(results_with_scores)}개 결과")
            print(f"{'='*80}")

        return results_with_scores 
    
    def print_results(self, results, top_n: int = None):
        """검색 결과 출력"""
        if top_n:
            results = results[:top_n]
        
        if not results:
            print("\n❌ 검색 결과가 없습니다.")
            return
        
        print(f"\n📊 검색 결과 (Top {len(results)}):")
        print("="*80)
        search_result = []
        for i, result in enumerate(results, 1):
            print(f"\n{i}. Score: {result['score']:.4f}")
            print(f"   Document ID: {result['payload']['doc_id']}")
            print(f"   지자체: {result['payload']['metadata'].get('지자체기관명', 'N/A')}")
            print(f"   법규명: {result['payload']['metadata'].get('자치법규명', 'N/A')}")
            
            links = result['payload']['metadata'].get('links', [])
            if links and links[0]:
                print(f"   관련 법령: {', '.join(list(links[0].keys())[:3])}")
            
            print(f"   내용: {result['payload']['page_content'][:200]}...")
            print("-"*80)
            search_result.append(result)
        return search_result
    
    def compare_weights(self, query: str, top_k: int = 3):
        """다양한 가중치로 검색 결과 비교"""
        print(f"\n{'='*80}")
        print(f"⚖️  가중치 비교: '{query}'")
        print(f"{'='*80}")
        
        weight_combinations = [
            (1.0, 0.0, "Dense only"),
            (0.0, 1.0, "Sparse only"),
            (0.8, 0.2, "Dense 우선"),
            (0.7, 0.3, "Dense 중심"),
            (0.5, 0.5, "균형"),
            (0.3, 0.7, "Sparse 중심"),
        ]
        
        for dense_w, sparse_w, desc in weight_combinations:
            print(f"\n{desc} (D:{dense_w}, S:{sparse_w})")
            results = self.search(
                query=query,
                top_k=top_k,
                dense_weight=dense_w,
                sparse_weight=sparse_w,
                verbose=False
            )
            
            for i, result in enumerate(results, 1):
                print(f"  {i}. {result['score']:.4f} | {result['payload']['doc_id']}")
                
# ============================================
# 3. 유틸리티 함수
# ============================================

def load_documents_from_jsonl(file_path: str) -> List[Document]:
    """JSONL 파일에서 Document 로드"""
    documents = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            data = json.loads(line)
            doc = Document(
                page_content=data["page_content"],
                metadata=data["metadata"],
                id=data.get("id", None)
            )
            documents.append(doc)
    return documents


In [2]:
import os
from langchain_community.embeddings import ClovaXEmbeddings

QDRANT_URL = os.getenv('QDRANT_URL', '')
QDRANT_api_key = os.getenv('QDRANT_api_key', '')

# Clova 임베더 초기화
clova_embedder = ClovaXEmbeddings(
                model="bge-m3",
                api_key=os.getenv("CLOVASTUDIO_API_KEY")
                )

# Qdrant 클라이언트 초기화
client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_api_key,
)
# 검색 엔진 생성
search_engine = HybridSearchEngine(
    clova_embedder=clova_embedder,
    client=client,
    collection_name="legal_docs_hybrid",
    bm25_encoder_file="kiwi_bm25_encoder.pkl"
)


🔧 HybridSearchEngine 초기화 완료


In [3]:
# 문서 로드
documents = load_documents_from_jsonl("documents/local/documents_merged.jsonl")

# # 인덱싱 (최초 1회)
# search_engine.index(documents, force_rebuild=False)

In [4]:
# JSON 파일 불러오기
with open("law_qa_dataset_100.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

### 1. 검색

In [5]:
intra_data = []
cross_data = []
for d in dataset:
    if d['type'] == "intra":
        intra_data.append(d)
    else:
        cross_data.append(d)

In [6]:
cross_data[12]

{'type': 'cross',
 'question': '수도법 제14조제3항이 규정하는 급수공사에 사용되는 자재의 기준은 무엇인가요?',
 'answer': '해당 지자체인 가평군에서는 급수공사에 사용되는 자재가 ‘수도법 제14조제3항 및 같은 법 시행령 제24조의2가 정한 기준에 적합해야 한다’고 규정하고 있습니다. 즉, 자재는 수도법이 정한 품질·안전 기준을 충족해야 하며, 이를 위반할 경우 공사 승인이 제한될 수 있습니다.',
 'legal_reference': '제9조(공사의 시행) 2항',
 'reasoning': '조례 제9조(공사의 시행) 2항에서 급수공사에 사용되는 자재는 수도법 제14조제3항 및 시행령 제24조의2의 기준에 적합해야 한다고 명시하고 있어, 해당 법 조문을 근거로 답변을 구성했습니다.',
 'link_reference': '수도법 제14조제3항',
 '자치법규ID': '2019827',
 '자치법규명': '가평군 수도급수 조례'}

In [7]:
# 검색
question = cross_data[12]['question']
results = search_engine.search(
    query=question,
    top_k=10,
    dense_weight=0.7,
    sparse_weight=0.3
)

# 결과 출력
search_output = search_engine.print_results(results)

📂 BM25 인코더 로드 중...
📂 BM25 인코더 로드: kiwi_bm25_encoder.json
✅ BM25 인코더 로드 완료 (Vocabulary: 2396)

🔍 검색 쿼리: '수도법 제14조제3항이 규정하는 급수공사에 사용되는 자재의 기준은 무엇인가요?'

1️⃣ Dense 벡터 생성 (Clova)...
  ✅ Dense 벡터: 1024 차원

2️⃣ Sparse 벡터 생성 (Kiwi BM25)...
  📝 형태소: ['수도', '법', '항', '규정', '급수', '공사', '사용', '자재', '기준']
  ✅ Sparse 벡터: 9 개 (non-zero)

3️⃣ Dense 검색 (가중치: 0.7)...


/var/folders/yn/6fdvg69n75b6lgybytxszz8c0000gn/T/ipykernel_45019/2297370966.py:496: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  dense_results = self.client.search(


  ✅ 30개 결과

4️⃣ Sparse 검색 (가중치: 0.3)...


/var/folders/yn/6fdvg69n75b6lgybytxszz8c0000gn/T/ipykernel_45019/2297370966.py:521: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  sparse_results = self.client.search(


  ✅ 30개 결과

5️⃣ RRF로 결과 결합...
  ✅ 최종 10개 결과

📊 검색 결과 (Top 10):

1. Score: 0.0886
   Document ID: 2022606_chapter1_chunk6
   지자체: 경상남도 거제시
   법규명: 거제시 수도급수 조례
   관련 법령: 기타
   내용: [경상남도 거제시 | 거제시 수도급수 조례]
제7조(공용급수설비의 설치 등) <개정 2009.12.30>① 시장은 공용급수설비의 설치 또는 사용을 허가할 수 있다. 다만, 그 비용은 수익자의 부담으로 한다. <개정 2009.12.30. 2016.11.29.>② <삭제 2016.11.29.>
제7조의2< 삭제 2009.12.30>
제7조의3< 삭제 2009....
--------------------------------------------------------------------------------

2. Score: 0.0856
   Document ID: 2025448_chapter2_chunk2
   지자체: 경상북도 경산시
   법규명: 경산시 수도급수 조례
   관련 법령: 기타, 수도법
   내용: [경상북도 경산시 | 경산시 수도급수 조례]
[제2장  급수공사의 비용]
제7조(공용급수장치의 설치등)① 시장이 필요하다고 인정하는 경우에는 공용 급수장치의 설치 또는 사용을 허가할 수 있다.② 제1항의 경우에 있어서 시장이 필요하다고 인정할 때에는 수익자금부담으로 이를 설치하고 사용하게 할 수 있다.
제8조(공사의 시행)① 급수공사의 설계 및 시공은 시장...
--------------------------------------------------------------------------------

3. Score: 0.0700
   Document ID: 2021640_chapter2_chunk2
   지자체: 전라남도 강진군
   법규명: 강진군 수도급수 조례
   관련 법령: 기타, 산업표준화법
   내용: [전라남도 강진군

In [152]:
search_output

[{'id': 687,
  'score': 0.08863636363636362,
  'payload': {'page_content': '[경상남도 거제시 | 거제시 수도급수 조례]\n제7조(공용급수설비의 설치 등) <개정 2009.12.30>① 시장은 공용급수설비의 설치 또는 사용을 허가할 수 있다. 다만, 그 비용은 수익자의 부담으로 한다. <개정 2009.12.30. 2016.11.29.>② <삭제 2016.11.29.>\n제7조의2< 삭제 2009.12.30>\n제7조의3< 삭제 2009.12.30>\n제8조(공사의 시행)① 급수공사의 설계(40밀리미터 미만) 및 시공은 시장이 시행한다. 다만, 시장이 직접 시공할 수 없을 때에는 시장이 따로 지정하는 급수공사 대행업자에게 그 공사의 전부 또는 일부를 시공하게 할 수 있다. <개정 2005. 6. 27>② 급수공사에 소요되는 자재는 한국산업규격 표시품으로 사용하여야 한다. <개정 2001.7.21, 2009.12.30, 2010.7.13>③ 제1항 단서 규정에 따른 공사에 있어서는 시장의 자재검사와 준공검사를 받아야 한다. 다만, 시공자재가 관급품일 때에는 그에 따른 시공자재 검사는 행하지 아니하며 급수공사 준공 후 정산한다. <개정 2009.12.30>④ 시장이 직접 급수공사를 시공할 경우에는 공사비 납부일부터 5일 이내에, 대행업자 수탁공사의 경우는 공사계약 후 3일 이내에 착공함을 원칙으로 한다. 다만, 천재지변 및 도로의 굴착 포장 또는 그 밖에 특별한 사유가 발생한 경우에는 그러하지 아니한다. <개정 2009.12.30,2014.12.1>⑤ 수탁받은 대행업자는 착공을 문서로 시장에게 통지하고, 소요되는 공사용 관급자재를 관급자재 청구서에 따라 시장에게 청구한다. <개정 2009.12.30., 개정 2018.5.10.>',
   'metadata': {'자치법규ID': '2022606',
    '자치법규명': '거제시 수도급수 조례',
    '자치법규일련번호': '2035591',
    '지자

In [155]:
def format_chunks_for_llm(chunks):
    formatted = ""
    for idx, c in enumerate(chunks):
        meta = c["payload"]["metadata"]
        content = c["payload"]["page_content"]
        content = content[content.find("]")+1:]
        links = meta.get("links", [])
        abbr = meta.get("약어", {})
        
        link_str = ""
        if links:
            link_str = "[참조 문서]"
            for d in links:
                for law, refs in d.items():
                    if law == "기타": law = "Attachment"
                    link_str += "\n- 'law_name' : '" + law + "'"
                    if refs:
                        link_str += ", 'clauses' : [" + ", ".join(refs)  + "]'"
                link_str += "\n"
            link_str = link_str.strip()


        # abbr_str = "[약어 정보]\n" + "\n- ".join([f"{k} = {v}" for k, v in abbr.items()]) if abbr else ""

        formatted += f"""
──────────────────────────────
[doc_id] {c["payload"]['doc_id']}
[자치법규명] {meta.get('자치법규명', '')}
[지자체기관명] {meta.get('지자체기관명', '')}
{content.strip()}

{link_str}
"""
    return formatted.strip()


In [174]:
chunks_for_llm = format_chunks_for_llm(search_output[3:5])
print(chunks_for_llm)

──────────────────────────────
[doc_id] 2019827_chapter2_chunk3
[자치법규명] 가평군 수도급수 조례
[지자체기관명] 경기도 가평군
[제2장 급수공사의 비용]
제8조(공용급수설비의 설치 등)① 군수가 필요하다고 인정하면 공용급수설비의 설치 또는 사용을 허가할 수 있다.<개정 2011.3.16.>② 제1항의 경우에 있어서 군수는 필요하다고 인정하면 수익자 부담으로 이를 설치하고 사용하게 할 수 있다.<개정 2011.3.16.>
제9조(공사의 시행)① 급수공사의 설계 및 시공은 군수가 수행한다. 다만, 군수가 직접 시공할 수 없을 경우에는 다음 각 호의 어느 하나에 해당하는 자를 지정하여 공사를 위탁시공하게 할 수 있다.<개정 2011.3.16.>1. 「건설산업기본법」제8조제1항에 따른 전문건설업자 중 상하수도설비 공사업자2. 군수가 지정하는 급수공사대행업자(이하 "시공업자"라 한다)② 급수공사에 소요되는 자재는 「수도법」제14조제3항 및 같은 법 시행령 제24조의2의 기준에 적합하여야 한다.<개정 2015.5.27.>③ 제1항 단서에 따른 공사에 있어서는 군수에게 시공자재 검사와 준공검사를 받아야 한다. 다만, 시공자재가 관급품일 경우에는 그에 따른 시공자재검사는 하지 아니한다.④ 급수공사는 군수가 직접 시공할 경우는 공사비 납부 일부터 5일 이내에, 시공업자가 시공할 경우는 공사계약 후 3일 이내에 착공함을 원칙으로 한다. 다만, 천재지변 및 도로의 굴착포장 또는 그 밖에 특별한 사유가 발생한 경우에는 예외로 한다.<개정 2015.5.27.>⑤ 시공업자는 착공을 군수에게 문서로 통보함과 동시에 관급자재 청구서를 군수에게 제출하여야 한다.

[참조 문서]
- 'law_name' : 'Attachment', 'clauses' : [제1항]'
- 'law_name' : '건설산업기본법', 'clauses' : [제8조제1항]'
- 'law_name' : '수도법', 'clauses' : [제14조제3항]'
- 'law

### 2. Ref Check

In [166]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

model = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0.1,
    # max_tokens = 4000,
    timeout=None,
    max_retries=2,
    reasoning = True,
)

system_message = """당신은 제공된 사용자 질문과 법령 문서의 Chunks를 보고 정확한 판단을 내리는 전문가이다.

각 Chunk는 실제 법령의 일부이며, [참조 문서] 섹션에는 이 조항에서 참조하고 있는 다른 법령명과 조항들이 제공되어 있다. 
또한, "별표", "서식"처럼 **같은 법령 내 첨부 문서(Attachment)** 도 포함된다.

# 아래의 원칙을 반드시 지키세요:
1. "is_relevant" — 해당 Chunk가 사용자의 질문에 근거가 되는가.
2. "needs_links" — 이 Chunk의 내용을 완전하게 이해하거나 구체적으로 답변하기 위해 [참조 문서]의 정보가 필요한가.
3. "needed_links" — [참조 문서]에 포함된 항목 중, 실제로 답변에 필요하다고 판단되는 문서나 조항 목록.
   - 반드시 [참조 문서]에 실제로 명시된 항목만 사용한다. (임의 생성 금지)
   - "Attachment" 항목 내 "별표", "서식"은 참조 문서로 간주한다.
   - "별표", "서식"은 외부 파일 형태의 문서로 존재하며, 이를 참조해야 실제 내용·수치·기준을 알 수 있다.

-----
아래는 사용자의 질문과 관련된 법령 문서의 Chunks이다.

[사용자 질문]
{user_query}

[Chunks]
{retrieved_contexts}

# 출력 규칙
- 출력은 반드시 **유효한 JSON 형식**으로 반환합니다.
- 각 요소는 다음 구조를 따라야 한다.
    "doc_id": "<doc_id>",
    "is_relevant": true/false,
    "needs_links": true/false,
    "needed_links": [
        JSON("law_name": "<참조 법령명>",
        "clauses": ["<제27조제2항>"]),
        JSON("law_name": "Attachment",
        "clauses": ["<별표>", "<서식>"]),
    ]

- [참조 문서]가 없는 경우, "needs_links"는 false를 반환한다.
- "clauses"는 문자열 배열로, 조항이 없을 경우 빈 배열([])을 반환한다.
- "clauses" 항목에 포함된 조항이나 항목이 현재 Chunk의 본문에 이미 존재한다면, "needed_links"에 추가하지 마십시오.
"""


prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("assistant", "{user_query}\n\n{retrieved_contexts}")
])

# 일반 Chain 생성
parser = JsonOutputParser()
chain = prompt | model | parser

chain

ChatPromptTemplate(input_variables=['retrieved_contexts', 'user_query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['retrieved_contexts', 'user_query'], input_types={}, partial_variables={}, template='당신은 제공된 사용자 질문과 법령 문서의 Chunks를 보고 정확한 판단을 내리는 전문가이다.\n\n각 Chunk는 실제 법령의 일부이며, [참조 문서] 섹션에는 이 조항에서 참조하고 있는 다른 법령명과 조항들이 제공되어 있다. \n또한, "별표", "서식"처럼 **같은 법령 내 첨부 문서(Attachment)** 도 포함된다.\n\n# 아래의 원칙을 반드시 지키세요:\n1. "is_relevant" — 해당 Chunk가 사용자의 질문에 근거가 되는가.\n2. "needs_links" — 이 Chunk의 내용을 완전하게 이해하거나 구체적으로 답변하기 위해 [참조 문서]의 정보가 필요한가.\n3. "needed_links" — [참조 문서]에 포함된 항목 중, 실제로 답변에 필요하다고 판단되는 문서나 조항 목록.\n   - 반드시 [참조 문서]에 실제로 명시된 항목만 사용한다. (임의 생성 금지)\n   - "Attachment" 항목 내 "별표", "서식"은 참조 문서로 간주한다.\n   - "별표", "서식"은 외부 파일 형태의 문서로 존재하며, 이를 참조해야 실제 내용·수치·기준을 알 수 있다.\n\n-----\n아래는 사용자의 질문과 관련된 법령 문서의 Chunks이다.\n\n[사용자 질문]\n{user_query}\n\n[Chunks]\n{retrieved_contexts}\n\n# 출력 규칙\n- 출력은 반드시 **유효한 JSON 형식**으로 반환합니다.\n- 각 요소는 

In [160]:
len(system_message), len(chunks_for_llm)

(1158, 4385)

In [173]:
question

'수도법 제14조제3항이 규정하는 급수공사에 사용되는 자재의 기준은 무엇인가요?'

In [181]:
ref_llm_check = chain.invoke({"user_query":question, "retrieved_contexts":chunks_for_llm})
if "results" in ref_llm_check:
    ref_llm_check = ref_llm_check["results"]
ref_llm_check

[{'doc_id': '2019827_chapter2_chunk3',
  'is_relevant': True,
  'needs_links': True,
  'needed_links': [{'law_name': '수도법', 'clauses': ['제14조제3항']},
   {'law_name': '수도법 시행령', 'clauses': ['제24조의2']}]},
 {'doc_id': '2021640_chapter2_chunk5',
  'is_relevant': False,
  'needs_links': False,
  'needed_links': []}]

In [182]:
ref_llm_check

[{'doc_id': '2019827_chapter2_chunk3',
  'is_relevant': True,
  'needs_links': True,
  'needed_links': [{'law_name': '수도법', 'clauses': ['제14조제3항']},
   {'law_name': '수도법 시행령', 'clauses': ['제24조의2']}]},
 {'doc_id': '2021640_chapter2_chunk5',
  'is_relevant': False,
  'needs_links': False,
  'needed_links': []}]

In [70]:
documents[0]

Document(id='2019610_chapter1_chunk1', metadata={'자치법규ID': '2019610', '자치법규명': '가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례', '자치법규일련번호': '1298118', '지자체기관명': '경기도 가평군', '시행일자': '20170731', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '2623', '전화번호': '031-580-2430', '담당부서명': '건설도시국 건설과', '자치법규발의종류': '', '공포일자': '20170731', 'links': [{'자연재해대책법': ['제27조제2항'], '도로법': [], '농어촌도로정비법': [], '자연재해대책법 시행령': ['제22조의8']}]}, page_content='[경기도 가평군 | 가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례]\n제1조(목적) 이 조례는 「자연재해대책법」 제27조제2항의 규정에 따라 건축물관리자의 제설·제빙에 관한 사항을 구체적으로 정함으로써 눈 또는 얼음으로 인한 주민의 불편을 최소화하고, 안전을 꾀함을 목적으로 한다.\n제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음과 같다. <개정 2017.7.31.>1. “도로”라 함은 「도로법」에 따른 도로, 그 밖에 일반 교통에 사용되는 모든 곳을 말한다.2. “차도”라 함은 연석선(차도와 보도를 구분하는 돌 등으로 이어진 선을 말한다), 안전표지, 그 밖에 이와 비슷한 공작물로써 그 경계를 표시하여 모든 차의 교통에 사용하도록 된 도로의 부분을 말한다.3. “보도(步道)”라 함은 연석선, 안전표지, 그 밖에 이와 비슷한 공작물로써 그 경계를 표시하여 보행자(유모차 및 신체장애인용 의자차를 포함한다)의 통행에 사용하도록 되어있는 도로의 부분을 말한다. <개정 2017.3.6.>4. “이면도로”라 함은 「도로법」에 따른 고속국도ㆍ일반국도ㆍ지방도ㆍ군도 및 농어촌도로(「농어촌도로정비법」에 따른 도로를 말한다)가 아닌 일

In [130]:
import re
import requests
import olefile
oc = os.getenv('OC', '')

def fetch_linked_content(law_name, clauses, doc_id, documents):
    """
    law_name: 참조 법령명 (예: '자연재해대책법' 또는 'Attachment')
    clauses: ['제27조제2항'] or ['별표']
    doc_id: 현재 chunk의 id (예: '2019610_chapter1_chunk3')
    documents: 전체 Document 리스트 (retrieval에 사용된)
    oc: 법령정보공개서비스(OpenAPI)용 OC key
    """

    results = []

    # 1️⃣ 일반 법령명 (다른 법령 참조)
    if law_name != "Attachment":
        for clause in clauses:
            for doc in documents:
                if law_name in doc.metadata["자치법규명"]:
                    if clause in doc.page_content:
                        results.append(f"📘 {law_name} {clause}\n{doc.page_content}")
                        break

    # 2️⃣ Attachment + “별표” / “서식” → 외부 문서 API 호출
    else:
        for clause in clauses:
            if any(keyword in clause for keyword in ["별표", "서식"]):
                # doc_id에서 숫자 ID 추출
                match = re.match(r"(\d+)_", doc_id)
                if match:
                    ID = match.group(1)
                    url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=ordin&ID={ID}&type=JSON"
                    try:
                        response = requests.get(url)
                        response.raise_for_status()
                        data = response.json()

                        annex_url = data["LawService"]["별표"]["별표단위"]["별표첨부파일명"]

                        annex_res = requests.get(annex_url)
                        annex_title = data["LawService"]["별표"]["별표단위"]["별표제목"]
                        with open(f"{annex_title}.hwp", "wb") as f:
                            f.write(annex_res.content)

                        f = olefile.OleFileIO(f"{annex_title}.hwp")  
                        encoded_text = f.openstream('PrvText').read() 
                        decoded_text = encoded_text.decode('utf-16')  
                        results.append(decoded_text.strip())

                    except Exception as e:
                        results.append("")
                        print(f"⚠️ 별표 문서 조회 실패: {e}")

    # 3️⃣ 결과 통합
    if results:
        return results
        # return "\n\n".join(results)
    else:
        print("📄 참조 문서를 찾을 수 없습니다.")
        return ""

In [183]:
fetch_linked_content("Attachment", ["별표"], "2022361_chapter1_chunk3", documents)

['[별표] <개정 2017.6.2.>\r\n지역별 지붕면의 제설·제빙 기준 적설량(제5조 관련)\r\n<건축구조기준(KBC 2015, 국토교통부) <그림 0304.2.2> 기본지상적설하중 (kN/m2)><적설량><제설・제빙 시점 적설량><비  고><기본지상적설하중 적용 환산 적설량 (cm)><특정 지형조건 (고지대, 산간지방) 가중치(1.5배) 적용 환산 적설량 (cm)><기본지상적설하중 적용 환산 적설량 (cm)><특정 지형조건 (고지대, 산간지방) 가중치(1.5배) 적용 환산 적설량 (cm)>\r\n<0.5이하 지역><50.0><75.0><25.0><37.5><>\r\n<0.5초과 1.0이하 지역><55.0><82.5><27.5><41.3>\r\n<1.0초과 1.5이하 지역><63.0><94.5><31.5><47.3>\r\n<1.5초과 2.0이하 지역 ><67.0><100.5><33.5><50.3>\r\n<2.0초과 2.5이하 지역><70.0><105.0><35.0><52.5>\r\n<2.5초과 3.0이하 지역><84.0><126.0><42.0><63.0>\r\n<3.0초과 3.5이하 지역><98.0><-><49.0><->\r\n<3.5초과 4.0이하 지역><112.0><-><56.0><->\r\n<4.0초과 4.5이하 지역><126.0><-><63.0><->\r\n<4.5초과 5.0이하 지역><140.0><-><70.0><->\r\n<5.0초과 지역><140.0><-><70.0><->\r\n<* 기상예보 강설량이 A~Bcm일 경우 제설·제빙 기준 예보강설량은 (A+B)/2로 본다. * 최소지상적설하중은 0.5kN/m2이상으로 한다.(KBC 2009)>\r\n       \r\n ∘ 기본지상적설하중의 경계에 있는 지역은 최고적설하중에 대한 값을 적용하고 기준값의 50%강설시 부터 제설·제빙작업을 실시하도록 제5조(제설·제빙시기)의 내용 표기']

In [193]:
for n, name in enumerate(df["법령명"]):
    if name == "수도법":
        print(df.iloc[n,:])
        break

순번              2991
법령MST         276757
소관부처코드       1482000
소관부처명       기후에너지환경부
법령ID            1818
법령명              수도법
공포일자        20251001
공포번호         제21065호
시행일자        20251001
법령구분코드         A0002
법령구분명             법률
법령분야코드      33030200
법령분야명     第2節 都市計劃施設
Name: 2990, dtype: object


In [195]:

id = df["법령ID"][n]
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID={id}&type=JSON"
response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

{'법령': {'개정문': {'개정문내용': [['⊙법률 제21065호(2025.10.1)', '정부조직법 일부개정법률', '[본문 생략]', '        부칙', '제1조(시행일) 이 법은 공포한 날부터 시행한다. 다만, 부칙 제7조에 따라 개정되는 법률 중 이 법 시행 전에 공포되었으나 시행일이 도래하지 아니한 법률을 개정한 부분은 각각 해당 법률의 시행일부터ㆍㆍㆍ<생략>ㆍㆍㆍ 시행한다.', '  1. 및 2. 생략', '제2조부터 제6조까지 생략', '제7조(다른 법률의 개정) ①부터 <353>까지 생략', '  <354> 수도법 일부를 다음과 같이 개정한다.', '  제3조제1호 단서, 같은 조 제7호 전단, 제4조제1항, 같은 조 제3항부터 제7항까지, 제5조제3항 전단, 같은 조 제5항, 제5조의2제1항 각 호 외의 부분 전단, 같은 항 제2호 본문, 같은 조 제2항 후단, 제6조제1항 전단, 같은 조 제3항 각 호 외의 부분 전단, 같은 조 제4항, 제7조제1항ㆍ제2항, 제7조의3제1항 각 호 외의 부분, 같은 조 제2항 전단, 제8조제3항, 제8조의2제2항ㆍ제3항, 제13조제2항, 제14조제1항ㆍ제4항ㆍ제6항ㆍ제7항, 제14조의2제1항 각 호 외의 부분 본문, 제14조의3제1항, 같은 조 제2항 각 호 외의 부분, 같은 조 제3항ㆍ제4항, 제14조의4제1항 각 호 외의 부분 본문, 같은 조 제2항, 제14조의5제1항 각 호 외의 부분, 같은 조 제2항ㆍ제3항, 제14조의6제1항 각 호 외의 부분, 같은 조 제2항, 같은 조 제3항 전단, 제14조의7제1항, 제14조의8제1항, 같은 조 제2항 본문, 제15조의2제1항 각 호 외의 부분, 같은 조 제2항, 제15조의3 각 호 외의 부분 본문, 제17조제1항 각 호 외의 부분 전단, 같은 항 제1호, 같은 조 제2항ㆍ제5항, 제21조의3제1항부터 제5항까지, 제21조의4제1항 각 호 외의 부분, 제21조의5제1항 각 호 외의 부분 본문, 제23조제2항 각 호 외의 부분, 제23조의2제1항부터 제3

In [199]:
data["법령"]["조문"]["조문단위"]

[{'조문번호': '1',
  '조문제개정유형': '타법개정',
  '조문시행일자': '20251001',
  '조문변경여부': 'N',
  '조문이동이전': '',
  '조문키': '0001000',
  '조문내용': '                    제1장 총칙',
  '조문제목': '',
  '조문이동이후': '',
  '조문여부': '전문'},
 {'조문번호': '1',
  '조문제개정유형': '타법개정',
  '조문시행일자': '20251001',
  '조문변경여부': 'N',
  '조문이동이전': '',
  '조문키': '0001001',
  '조문내용': '제1조(목적) 이 법은 수도(水道)에 관한 종합적인 계획을 수립하고 수도를 적정하고 합리적으로 설치ㆍ관리하여 공중위생을 향상시키고 생활환경을 개선하게 하는 것을 목적으로 한다.',
  '조문제목': '목적',
  '조문이동이후': '',
  '조문여부': '조문'},
 {'조문번호': '2',
  '조문제개정유형': '타법개정',
  '조문시행일자': '20251001',
  '조문변경여부': 'N',
  '조문이동이전': '',
  '조문키': '0002001',
  '항': [{'항번호': '①',
    '항내용': '①국가는 모든 국민이 질 좋은 물을 공급받을 수 있도록 수도에 관한 종합적인 계획을 수립하고 합리적인 시책을 강구하며 수도사업자에 대한 기술 지원 및 재정 지원을 위하여 노력하여야 한다.'},
   {'항번호': '②',
    '항내용': '②특별시장ㆍ광역시장ㆍ특별자치시장ㆍ도지사ㆍ특별자치도지사(이하 "시ㆍ도지사"라 한다)와 시장ㆍ군수ㆍ구청장(자치구의 구청장을 말한다. 이하 같다)은 관할 구역의 주민이 질 좋은 물을 공급받을 수 있도록 상수원의 관리 등에 노력하여야 한다. <개정 2010.5.25, 2011.11.14>'},
   {'항번호': '③',
    '항내용': '③특별시장ㆍ광역시장ㆍ특별자치시장ㆍ특별자치도지사ㆍ시장ㆍ군수(광역시의 군수는 제외한다)ㆍ상수도조합은 관

In [184]:
fetch_linked_content("수도법", ['제14조제3항'], "2022361_chapter1_chunk3", documents)

📄 참조 문서를 찾을 수 없습니다.


''

In [185]:
documents

[Document(id='2019610_chapter1_chunk1', metadata={'자치법규ID': '2019610', '자치법규명': '가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례', '자치법규일련번호': '1298118', '지자체기관명': '경기도 가평군', '시행일자': '20170731', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '2623', '전화번호': '031-580-2430', '담당부서명': '건설도시국 건설과', '자치법규발의종류': '', '공포일자': '20170731', 'links': [{'자연재해대책법': ['제27조제2항'], '도로법': [], '농어촌도로정비법': [], '자연재해대책법 시행령': ['제22조의8']}]}, page_content='[경기도 가평군 | 가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례]\n제1조(목적) 이 조례는 「자연재해대책법」 제27조제2항의 규정에 따라 건축물관리자의 제설·제빙에 관한 사항을 구체적으로 정함으로써 눈 또는 얼음으로 인한 주민의 불편을 최소화하고, 안전을 꾀함을 목적으로 한다.\n제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음과 같다. <개정 2017.7.31.>1. “도로”라 함은 「도로법」에 따른 도로, 그 밖에 일반 교통에 사용되는 모든 곳을 말한다.2. “차도”라 함은 연석선(차도와 보도를 구분하는 돌 등으로 이어진 선을 말한다), 안전표지, 그 밖에 이와 비슷한 공작물로써 그 경계를 표시하여 모든 차의 교통에 사용하도록 된 도로의 부분을 말한다.3. “보도(步道)”라 함은 연석선, 안전표지, 그 밖에 이와 비슷한 공작물로써 그 경계를 표시하여 보행자(유모차 및 신체장애인용 의자차를 포함한다)의 통행에 사용하도록 되어있는 도로의 부분을 말한다. <개정 2017.3.6.>4. “이면도로”라 함은 「도로법」에 따른 고속국도ㆍ일반국도ㆍ지방도ㆍ군도 및 농어촌도로(「농어촌도로정비법」에 따른 도로를 말한다)가 아닌 

### 3. 답변 생성

In [90]:
search_chunk = search_output[0]
search_chunk

{'id': 2,
 'score': 0.08863636363636362,
 'payload': {'page_content': '[경기도 가평군 | 가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례]\n제6조(제설ㆍ제빙작업의 시기) 건축물관리자는 제설ㆍ제빙작업을 눈이 그친 때부터 3시간 이내에 완료하여야 한다. 다만, 야간(일몰 후부터 다음 날 일출 전까지)에 눈이 내린 때에는 다음 날 오전 11시까지 제설ㆍ제빙작업을 완료하여야 한다.\n제7조(제설ㆍ제빙작업의 방법) 건축물관리자는 보행자나 차량의 안전한 통행에 지장이 없도록 삽, 빗자루 등의 작업도구를 이용하여 다음 각 호와 같이 제설ㆍ제빙작업을 하여야 한다.1. 보도ㆍ이면도로 및 보행자전용도로의 눈이나 얼음 : 도로의 가장자리나 공터 등으로 옮긴다.2. 얼음 제거가 어려운 경우에는 얼음을 녹게 하는 재료나 모래 등을 사용하고, 얼음이 녹은 후에는 뿌려진 모래 등을 깨끗이 제거한다.3. 시설물의 지붕에서 발생되는 눈은 시설물의 대지 안에 옮긴다. 다만, 대지 안에 여유공간이 없을 경우 보도의 가장자리나 공터 등으로 옮긴다. <신설 2017.7.31.>',
  'metadata': {'자치법규ID': '2019610',
   '자치법규명': '가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례',
   '자치법규일련번호': '1298118',
   '지자체기관명': '경기도 가평군',
   '시행일자': '20170731',
   '제개정정보': '일부개정',
   '자치법규종류': 'C0001',
   '공포번호': '2623',
   '전화번호': '031-580-2430',
   '담당부서명': '건설도시국 건설과',
   '자치법규발의종류': '',
   '공포일자': '20170731'},
  'doc_id': '2019610_chapter1_chunk3'}}

In [88]:
ref = ref_llm_check[0]
ref

{'doc_id': '2019610_chapter1_chunk3',
 'is_relevant': True,
 'needs_links': False,
 'needed_links': []}

In [136]:
def related_chunks_for_answer(ref_llm_check, search_output):
    final_docs = []
    formatted = ""
    for ref, search_chunk in zip(ref_llm_check, search_output):
        if ref["is_relevant"]:
            # formatted = ""
            meta = search_chunk["payload"]["metadata"]
            content = search_chunk["payload"]["page_content"]
            content = content[content.find("]")+1:]
            links = meta.get("links", [])
            abbr = meta.get("약어", {})
            abbr_str = "[약어 정보]\n" + "\n- ".join([f"{k} = {v}" for k, v in abbr.items()]) if abbr else ""
            link_str = ""
            new_link = {}
            if ref["needs_links"]:
                for link in ref["needed_links"]:
                    if link["law_name"] == "Attachment":
                        cc  = link["clauses"]
                        print(f"== {cc} 처리 중 ==")
                        try:
                            # print(link["law_name"], link["clauses"], ref["doc_id"],)
                            clause_link = fetch_linked_content(link["law_name"], link["clauses"], ref["doc_id"], documents)
                            for c, clause_txt in enumerate(clause_link):
                                if clause_txt:
                                    link_str = "[별표 내용]\n" + clause_txt
                                    new_link[link["law_name"]] = {"clauses": link["clauses"][c], "content": clause_txt}
                                # print(clause_txt)
                                print(f"== {cc} 처리 완료 ==")
                        except Exception as e: # 같은 문서 다른 조항 | 같은 문서 같은 조항
                            print(f"⚠️ 오류 발생: {e}")
                    else: # 외부 문서
                        pass
            search_chunk["payload"]["metadata"]["link_contents"] = []
            if new_link:
                search_chunk["payload"]["metadata"]["link_contents"].append(new_link)
            final_docs.append(search_chunk)
            formatted += f"""[자치법규명] {meta.get('자치법규명', '')}
[지자체기관명] {meta.get('지자체기관명', '')}
{content.strip()}

{link_str}
{abbr_str}
------------------------------
"""
    return formatted, final_docs
# print(related_chunks_for_answer(ref_llm_check, search_output))

In [137]:
final_chunks, final_docs = related_chunks_for_answer(ref_llm_check, search_output)

== ['별표'] 처리 중 ==
== ['별표'] 처리 완료 ==
== ['제7조'] 처리 중 ==
📄 참조 문서를 찾을 수 없습니다.


In [113]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0.7,
    max_tokens = 2048,
    timeout=None,
    max_retries=2,
    reasoning = None,
)

system_message = """당신은 제공된 법률 문서의 일부를 기반으로 사용자의 질문에 근거를 기반으로 구체적으로 답변하세요.

[법률 문서]
{final_chunks}

"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("assistant", "{final_chunks}"),
    ("user", "{question}")
])

chain = prompt | model

# 일반 Chain 생성
chain = prompt | model | StrOutputParser()
chain

ChatPromptTemplate(input_variables=['final_chunks', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['final_chunks'], input_types={}, partial_variables={}, template='당신은 제공된 법률 문서의 일부를 기반으로 사용자의 질문에 근거를 기반으로 구체적으로 답변하세요.\n\n[법률 문서]\n{final_chunks}\n\n'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['final_chunks'], input_types={}, partial_variables={}, template='{final_chunks}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOllama(model='gpt-oss:120b-cloud', temperature=0.7)
| StrOutputParser()

In [138]:
# 최종 ref
final_docs

[{'id': 2,
  'score': 0.08863636363636362,
  'payload': {'page_content': '[경기도 가평군 | 가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례]\n제6조(제설ㆍ제빙작업의 시기) 건축물관리자는 제설ㆍ제빙작업을 눈이 그친 때부터 3시간 이내에 완료하여야 한다. 다만, 야간(일몰 후부터 다음 날 일출 전까지)에 눈이 내린 때에는 다음 날 오전 11시까지 제설ㆍ제빙작업을 완료하여야 한다.\n제7조(제설ㆍ제빙작업의 방법) 건축물관리자는 보행자나 차량의 안전한 통행에 지장이 없도록 삽, 빗자루 등의 작업도구를 이용하여 다음 각 호와 같이 제설ㆍ제빙작업을 하여야 한다.1. 보도ㆍ이면도로 및 보행자전용도로의 눈이나 얼음 : 도로의 가장자리나 공터 등으로 옮긴다.2. 얼음 제거가 어려운 경우에는 얼음을 녹게 하는 재료나 모래 등을 사용하고, 얼음이 녹은 후에는 뿌려진 모래 등을 깨끗이 제거한다.3. 시설물의 지붕에서 발생되는 눈은 시설물의 대지 안에 옮긴다. 다만, 대지 안에 여유공간이 없을 경우 보도의 가장자리나 공터 등으로 옮긴다. <신설 2017.7.31.>',
   'metadata': {'자치법규ID': '2019610',
    '자치법규명': '가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례',
    '자치법규일련번호': '1298118',
    '지자체기관명': '경기도 가평군',
    '시행일자': '20170731',
    '제개정정보': '일부개정',
    '자치법규종류': 'C0001',
    '공포번호': '2623',
    '전화번호': '031-580-2430',
    '담당부서명': '건설도시국 건설과',
    '자치법규발의종류': '',
    '공포일자': '20170731',
    'link_contents': []},
   'doc_id': '2019610_chapter1_chunk3'}},
 {'id': 372,
  'score': 0.0814102564

In [140]:
print(final_chunks)

[자치법규명] 가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례
[지자체기관명] 경기도 가평군
제6조(제설ㆍ제빙작업의 시기) 건축물관리자는 제설ㆍ제빙작업을 눈이 그친 때부터 3시간 이내에 완료하여야 한다. 다만, 야간(일몰 후부터 다음 날 일출 전까지)에 눈이 내린 때에는 다음 날 오전 11시까지 제설ㆍ제빙작업을 완료하여야 한다.
제7조(제설ㆍ제빙작업의 방법) 건축물관리자는 보행자나 차량의 안전한 통행에 지장이 없도록 삽, 빗자루 등의 작업도구를 이용하여 다음 각 호와 같이 제설ㆍ제빙작업을 하여야 한다.1. 보도ㆍ이면도로 및 보행자전용도로의 눈이나 얼음 : 도로의 가장자리나 공터 등으로 옮긴다.2. 얼음 제거가 어려운 경우에는 얼음을 녹게 하는 재료나 모래 등을 사용하고, 얼음이 녹은 후에는 뿌려진 모래 등을 깨끗이 제거한다.3. 시설물의 지붕에서 발생되는 눈은 시설물의 대지 안에 옮긴다. 다만, 대지 안에 여유공간이 없을 경우 보도의 가장자리나 공터 등으로 옮긴다. <신설 2017.7.31.>



------------------------------
[자치법규명] 강진군 건축물 관리자의 제설 및 제빙책임에 관한 조례
[지자체기관명] 전라남도 강진군
제6조(제설·제빙작업의 시기) 건축물관리자는 제설·제빙작업을 눈이 그친 때로부터 3시간 이내에 완료하여야 한다. 다만, 야간(일몰 후부터 다음날 일출 전까지)에 눈이 내린 경우에는 다음날 오전 10시까지 제설·제빙작업을 완료하여야 한다.
제7조(제설·제빙작업의 방법)① 도로상의 눈이나 얼음은 삽, 빗자루 등의 작업도구로 보행자나 차량의 안전한 통행에 지장이 없도록 도로의 가장자리나 공터 등으로 옮겨 쌓아야 한다.② 도로상의 얼음 제거가 어려운 경우에는 얼음을 녹게 하는 재료나 모래 등을 뿌려서 보행자나 차량의 안전한 통행에 지장이 없도록 하여야 하고, 얼음이 녹은 후에는 뿌려진 모래 등을 제거하여 도로를 깨끗하게 하여야 한다.
제8조(제설·제빙작업의 도구 비치·관리) 건축물관리자는 보도·이면도로 및 보행자 

In [111]:
answer = chain.invoke({"question":question, "final_chunks": final_chunks})
print(answer)

각 지방자치단체마다 제설·제빙 작업을 완료해야 하는 시점이 다소 차이가 있습니다. 아래에 제시된 **모든 조례**를 근거로 정리하면 다음과 같습니다.

| 지방자치단체 (조례명) | 눈이 그친 뒤 작업 완료 시한 | 야간(일몰‑일출) 강설 시 별도 시한 |
|----------------------|----------------------------|-----------------------------------|
| **가평군**<br>《가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례》 | 눈이 그친 때부터 **3시간 이내** | 다음 날 **오전 11시**까지 |
| **강진군**<br>《강진군 건축물 관리자의 제설 및 제빙책임에 관한 조례》 | 눈이 그친 때부터 **3시간 이내** | 다음 날 **오전 10시**까지 |
| **경산시**<br>《경산시 건축물관리자의 제설 및 제빙 책임에 관한 조례》 | 눈이 그친 때부터 **4시간 이내** | 다음 날 **오전 12시(정오)**까지 |
| **거제시**<br>《거제시 건축물관리자의 제설·제빙에 관한 조례》 | - 주간: 눈이 그친 때부터 **4시간 이내**<br>- 야간: 다음 날 **오전 11시**까지<br>※ 단, 1일 강설량이 10 cm 이상이면 눈이 그친 때부터 **24시간 이내** |
| **강릉시**<br>《강릉시 건축물관리자의 제설 및 제빙 책임에 관한 조례》 | 눈이 그친 때부터 **4시간 이내** | 다음 날 **낮 12시(정오)**까지 |

### 요약
- **일반적인(주간) 경우**  
  - 가평·강진군은 **3시간**,  
  - 경산·거제·강릉시는 **4시간** 이내에 제설·제빙 작업을 마쳐야 합니다.  

- **야간에 눈이 내린 경우**(일몰‑일출 구간)  
  - 가평군 → **오전 11시**까지  
  - 강진군 → **오전 10시**까지  
  - 경산시 → **오전 12시(정오)**까지  
  - 거제시 → **오전 11시**까지 (단, 10 cm 이상 강설 시 2

In [114]:
answer = chain.invoke({"question":question, "final_chunks": final_chunks})
print(answer)

아래는 질문에 포함된 **각 자치법규(조례)** 별로 **‘눈이 그친 뒤 건축물관리자가 제설·제빙 작업을 완료해야 하는 시점’**을 정리한 내용입니다.  

| 지방자치단체(조례) | 기본(주간) 완료 시한 | 야간(일몰‑일출) 완료 시한 | 비고 |
|-------------------|----------------------|---------------------------|------|
| **가평군** (경기도) | 눈이 그친 때부터 **3시간 이내** | 다음 날 **오전 11시**까지 | ① 주간은 3시간, ② 야간은 오전 11시 (제6조) |
| **강진군** (전라남도) | 눈이 그친 때부터 **3시간 이내** | 다음 날 **오전 10시**까지 | ① 주간·야간 모두 3시간 기준, 다만 야간은 오전 10시 (제6조) |
| **경산시** (경상북도) | 눈이 그친 때부터 **4시간 이내** | 다음 날 **오전 12시(정오)**까지 | ① 주간·야간 모두 4시간 기준, 야간은 정오까지 (제6조) |
| **거제시** (경상남도) | 눈이 그친 때부터 **주간 4시간 이내** | 다음 날 **오전 11시**까지 (단, 1일 적설량 ≥ 10 cm이면 **눈이 그친 시점부터 24시간 이내**) | ① 주간 4시간, ② 야간 오전 11시 (제5조) |
| **강릉시** (강원도) | 눈이 그친 때부터 **4시간 이내** | 다음 날 **낮 12시(정오)**까지 | ① 주간·야간 모두 4시간 기준, 야간은 정오까지 (제5조) |

### 핵심 정리
1. **주간(일몰 전)**에 눈이 그친 경우  
   - 가평·강진: **3시간** 이내  
   - 경산·거제·강릉: **4시간** 이내  

2. **야간(일몰 후‑일출 전)**에 눈이 그친 경우(또는 눈이 내린 시점이 야간인 경우)  
   - 가평: **다음 날 오전 11시**까지  
   - 강진: **다음 날 오전 10시**까지  
   - 경산·강릉: **다음 날 정오(12시)**까지  
   

In [135]:
import requests
oc = os.getenv('OC', '')
url = f"https://www.law.go.kr/DRF/lawSearch.do?OC={oc}&target=ordinbyl&type=JSON"
response = requests.get(url)
response.raise_for_status()

data = response.json()

In [55]:
"2022361" in data

False

In [56]:
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=ordin&ID={ID}&type=JSON"
response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

{'LawService': {'자치법규기본정보': {'자치법규ID': '2022361', '자치법규명': '거제시 건축물관리자의 제설·제빙에 관한 조례', '자치법규일련번호': '1559741', '지자체기관명': '경상남도 거제시', '시행일자': '20201229', '제개정정보': '일부개정', '자치법규종류': 'C0001', '공포번호': '1786', '전화번호': '055-639-3692', '담당부서명': '재난대응과', '자치법규발의종류': '', '공포일자': '20201229'}, '별표': {'별표단위': {'별표제목': '별표(지역별 지붕면의 제설.제빙 기준 적설량)', '별표첨부파일구분': 'hwp', '별표첨부파일명': 'http://www.law.go.kr/flDownload.do?gubun=ELIS&flSeq=157494151&flNm=%EB%B3%84%ED%91%9C%28%EC%A7%80%EC%97%AD%EB%B3%84+%EC%A7%80%EB%B6%95%EB%A9%B4%EC%9D%98+%EC%A0%9C%EC%84%A4.%EC%A0%9C%EB%B9%99+%EA%B8%B0%EC%A4%80+%EC%A0%81%EC%84%A4%EB%9F%89%29', '별표번호': '0001', '별표키': '20989125', '별표내용': '', '별표구분': '서식', '별표가지번호': '00'}}, '부칙': {'부칙공포일자': '20201229', '부칙내용': '부  칙  이 조례는 공포한 날부터 시행한다.부      칙 <2017.6.2. 조례 제1532호> 이 조례는 공포한 날부터 시행한다.부      칙 <2017.11.9. 조례 제1559호> 이 조례는 공포한 날부터 시행한다.부      칙 <2020.12.29. 조례 제1786호> 이 조례는 공포한 날부터 시행한다.', '부칙공포번호': '1786'}, '조문': {'조': [{'조문번호': ['000100', '000100'], '조제목': '목적', '조내용': '제1조(목적) 

In [57]:
data["LawService"]["별표"]

{'별표단위': {'별표제목': '별표(지역별 지붕면의 제설.제빙 기준 적설량)',
  '별표첨부파일구분': 'hwp',
  '별표첨부파일명': 'http://www.law.go.kr/flDownload.do?gubun=ELIS&flSeq=157494151&flNm=%EB%B3%84%ED%91%9C%28%EC%A7%80%EC%97%AD%EB%B3%84+%EC%A7%80%EB%B6%95%EB%A9%B4%EC%9D%98+%EC%A0%9C%EC%84%A4.%EC%A0%9C%EB%B9%99+%EA%B8%B0%EC%A4%80+%EC%A0%81%EC%84%A4%EB%9F%89%29',
  '별표번호': '0001',
  '별표키': '20989125',
  '별표내용': '',
  '별표구분': '서식',
  '별표가지번호': '00'}}

In [131]:
data["LawService"]["별표"]["별표단위"]["별표첨부파일명"]

NameError: name 'data' is not defined

In [60]:
url = "http://www.law.go.kr/flDownload.do?gubun=ELIS&flSeq=157494151&flNm=%EB%B3%84%ED%91%9C%28%EC%A7%80%EC%97%AD%EB%B3%84+%EC%A7%80%EB%B6%95%EB%A9%B4%EC%9D%98+%EC%A0%9C%EC%84%A4.%EC%A0%9C%EB%B9%99+%EA%B8%B0%EC%A4%80+%EC%A0%81%EC%84%A4%EB%9F%89%29"
res = requests.get(url)
with open("별표_지붕면_제설기준.hwp", "wb") as f:
    f.write(res.content)

In [61]:
import olefile
f = olefile.OleFileIO('별표_지붕면_제설기준.hwp')  
encoded_text = f.openstream('PrvText').read() 
decoded_text = encoded_text.decode('utf-16')  
decoded_text

' [별표] <개정 2017.6.2.>\r\n지역별 지붕면의 제설·제빙 기준 적설량(제5조 관련)\r\n<건축구조기준(KBC 2015, 국토교통부) <그림 0304.2.2> 기본지상적설하중 (kN/m2)><적설량><제설・제빙 시점 적설량><비  고><기본지상적설하중 적용 환산 적설량 (cm)><특정 지형조건 (고지대, 산간지방) 가중치(1.5배) 적용 환산 적설량 (cm)><기본지상적설하중 적용 환산 적설량 (cm)><특정 지형조건 (고지대, 산간지방) 가중치(1.5배) 적용 환산 적설량 (cm)>\r\n<0.5이하 지역><50.0><75.0><25.0><37.5><>\r\n<0.5초과 1.0이하 지역><55.0><82.5><27.5><41.3>\r\n<1.0초과 1.5이하 지역><63.0><94.5><31.5><47.3>\r\n<1.5초과 2.0이하 지역 ><67.0><100.5><33.5><50.3>\r\n<2.0초과 2.5이하 지역><70.0><105.0><35.0><52.5>\r\n<2.5초과 3.0이하 지역><84.0><126.0><42.0><63.0>\r\n<3.0초과 3.5이하 지역><98.0><-><49.0><->\r\n<3.5초과 4.0이하 지역><112.0><-><56.0><->\r\n<4.0초과 4.5이하 지역><126.0><-><63.0><->\r\n<4.5초과 5.0이하 지역><140.0><-><70.0><->\r\n<5.0초과 지역><140.0><-><70.0><->\r\n<* 기상예보 강설량이 A~Bcm일 경우 제설·제빙 기준 예보강설량은 (A+B)/2로 본다. * 최소지상적설하중은 0.5kN/m2이상으로 한다.(KBC 2009)>\r\n       \r\n ∘ 기본지상적설하중의 경계에 있는 지역은 최고적설하중에 대한 값을 적용하고 기준값의 50%강설시 부터 제설·제빙작업을 실시하도록 제5조(제설·제빙시기)의 내용 표기\r\n\r\n'

In [62]:
len(decoded_text)

906

In [63]:
links_chunk_id = [4]

In [64]:
def format_chunks_for_llm(chunks, links_chunk_id):
    formatted = ""
    for idx, c in enumerate(chunks):
        meta = c["payload"]["metadata"]
        content = c["payload"]["page_content"]
        content = content[content.find("]")+1:]
        links = meta.get("links", [])
        abbr = meta.get("약어", {})

        link_text = []
        for d in links:
            for law, refs in d.items():
                for r in refs:
                    link_text.append(f"{law} {r}")
        link_str = ""
        if idx in links_chunk_id:
            link_str = "[별표 내용]\n" + decoded_text
        abbr_str = "[약어 정보]\n" + "\n- ".join([f"{k} = {v}" for k, v in abbr.items()]) if abbr else ""

        formatted += f"""
──────────────────────────────
# Chunk {idx+1}
[자치법규명] {meta.get('자치법규명', '')}
[지자체기관명] {meta.get('지자체기관명', '')}
{content.strip()}

{link_str}
{abbr_str}
"""
    return formatted.strip()


In [68]:
final_chunks = format_chunks_for_llm(search_output[:5], links_chunk_id)
print(final_chunks)

──────────────────────────────
# Chunk 1
[자치법규명] 가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례
[지자체기관명] 경기도 가평군
제6조(제설ㆍ제빙작업의 시기) 건축물관리자는 제설ㆍ제빙작업을 눈이 그친 때부터 3시간 이내에 완료하여야 한다. 다만, 야간(일몰 후부터 다음 날 일출 전까지)에 눈이 내린 때에는 다음 날 오전 11시까지 제설ㆍ제빙작업을 완료하여야 한다.
제7조(제설ㆍ제빙작업의 방법) 건축물관리자는 보행자나 차량의 안전한 통행에 지장이 없도록 삽, 빗자루 등의 작업도구를 이용하여 다음 각 호와 같이 제설ㆍ제빙작업을 하여야 한다.1. 보도ㆍ이면도로 및 보행자전용도로의 눈이나 얼음 : 도로의 가장자리나 공터 등으로 옮긴다.2. 얼음 제거가 어려운 경우에는 얼음을 녹게 하는 재료나 모래 등을 사용하고, 얼음이 녹은 후에는 뿌려진 모래 등을 깨끗이 제거한다.3. 시설물의 지붕에서 발생되는 눈은 시설물의 대지 안에 옮긴다. 다만, 대지 안에 여유공간이 없을 경우 보도의 가장자리나 공터 등으로 옮긴다. <신설 2017.7.31.>




──────────────────────────────
# Chunk 2
[자치법규명] 강진군 건축물 관리자의 제설 및 제빙책임에 관한 조례
[지자체기관명] 전라남도 강진군
제6조(제설·제빙작업의 시기) 건축물관리자는 제설·제빙작업을 눈이 그친 때로부터 3시간 이내에 완료하여야 한다. 다만, 야간(일몰 후부터 다음날 일출 전까지)에 눈이 내린 경우에는 다음날 오전 10시까지 제설·제빙작업을 완료하여야 한다.
제7조(제설·제빙작업의 방법)① 도로상의 눈이나 얼음은 삽, 빗자루 등의 작업도구로 보행자나 차량의 안전한 통행에 지장이 없도록 도로의 가장자리나 공터 등으로 옮겨 쌓아야 한다.② 도로상의 얼음 제거가 어려운 경우에는 얼음을 녹게 하는 재료나 모래 등을 뿌려서 보행자나 차량의 안전한 통행에 지장이 없도록 하여야 하고, 얼음이 녹은 후에는 뿌려진 모래 등을 제거하여 도로를 깨끗하게 하

In [69]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0.7,
    max_tokens = 2048,
    timeout=None,
    max_retries=2,
    reasoning = None,
)

system_message = """당신은 제공된 법률 문서의 일부를 기반으로 사용자의 질문에 근거를 기반으로 구체적으로 답변하세요.

[법률 문서]
{final_chunks}

"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{question}")
])

chain = prompt | model

# 일반 Chain 생성
chain = prompt | model | StrOutputParser()
chain

ChatPromptTemplate(input_variables=['final_chunks', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['final_chunks'], input_types={}, partial_variables={}, template='당신은 제공된 법률 문서의 일부를 기반으로 사용자의 질문에 근거를 기반으로 구체적으로 답변하세요.\n\n[법률 문서]\n{final_chunks}\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOllama(model='gpt-oss:120b-cloud', temperature=0.7)
| StrOutputParser()

In [70]:
question

'건축물관리자는 눈이 그친 뒤 제설·제빙 작업을 언제까지 완료해야 하나요?'

In [72]:
answer = chain.invoke({"question":question, "final_chunks": final_chunks})
print(answer)

아래는 제공해 주신 조례 5개(가평군·강진군·경산시·거제시·강릉시) 각각에서 규정하고 있는 **‘눈이 그친 뒤 제설·제빙 작업을 언제까지 완료해야 하는가’**에 대한 요약입니다.  

| 지자체(조례) | 기본 완료 시한 | 야간(일몰 ~ 다음날 일출) 눈이 내렸을 경우 추가 시한 | 특수·예외 규정 |
|--------------|----------------|--------------------------------------------|----------------|
| **가평군**<br>※ 가평군 건축물관리자의 제설·제빙 책임에 관한 조례 (제6조) | 눈이 그친 **3시간 이내** | **다음날 오전 11시**까지 | – |
| **강진군**<br>※ 강진군 건축물 관리자의 제설·제빙책임에 관한 조례 (제6조) | 눈이 그친 **3시간 이내** | **다음날 오전 10시**까지 | – |
| **경산시**<br>※ 경산시 건축물관리자의 제설·제빙 책임에 관한 조례 (제6조) | 눈이 그친 **4시간 이내** | **다음날 정오(12시)**까지 | – |
| **거제시**<br>※ 거제시 건축물관리자의 제설·제빙에 관한 조례 (제5조) | *보도·이면도로·보행자전용도로*<br>• 주간: **4시간 이내**<br>• 야간: **다음날 오전 11시**까지 | **다음날 오전 11시** (야간) | ① 눈이 10 cm 이상 내린 경우 → 눈이 그친 시점부터 **24시간 이내**<br>② 지붕에 눈이 쌓여 시설물 안전에 영향을 줄 정도이면 **즉시** 제거 |
| **강릉시**<br>※ 강릉시 건축물관리자의 제설·제빙 책임에 관한 조례 (제5조) | 눈이 그친 **4시간 이내** | **다음날 정오(12시)**까지 | – |

### 핵심 정리
1. **일반(주간) 기준**  
   - 가평·강진: **3시간** 내  
   - 경산·거제·강릉: **4시간** 내  

2. **야간(일몰 ~ 다음날 일출) 기준**  
   - 가평: **다음날 11시**  